---
### RSA Workbook
---

CSC424: Computer Networks, semester 252
<br>
University of Miami
<br>
Burton Rosenberg
<br>
6 April 2025
<br>
(c) 2025 Burton Rosenberg; All rights reserved

---

### Number Theory primitives.

This class gives some helpful functions. 

The extended GCD is the solution $x, y$ given $a, b$ of,
$$
\gcd(a,b) = x\,a + u\,b
$$
fover the integers. This algorithm has various uses,

- to calculate the greatest common divisor of two integers
- to check for relative primality of two integers
- to quickly calculate inverses for a integer ring.

We give here a quick modular exponentiation based on repeated squarings.

We give a brute force method to check if a number is prime.


In [12]:
import math
import random


class NumTheory:
    
    @staticmethod
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = NumTheory.egcd(b % a, a)
            return (g, x - (b // a) * y, y)

    @staticmethod
    def modinv(a, m):
        g, x, y = NumTheory.egcd(a, m)
        if g != 1:
            raise Exception('modular inverse does not exist')
        else:
            return x % m

    @staticmethod
    def modpow(a,i,m):
        assert i>=0, 'power cannot be negative'
        a = a%m
        if i==0:
            return 1
        if i%2==1:
            return (a*NumTheory.modpow(a,i-1,m)) % m
        c = NumTheory.modpow(a,i/2,m)
        return c*c % m
        
        
    @staticmethod
    def is_prime(n):
        if n < 2: return False
        if n % 2 == 0: return False
        k = 3
        while k*k <= n:
            if n % k == 0: return False
            k += 2
        return True

test = True
#test = False
if test:
    print(f'gcd(3,7): {NumTheory.egcd(3,7)[0]}')
    print(f'gcd(21,7): {NumTheory.egcd(21,7)[0]}')
    print(f'(1/3) mod 7: {NumTheory.modinv(3,7)}')
    print(f'3**3 mod 7: {NumTheory.modpow(3,3,7)}, {3**3 % 7}' )
    print(f'4**4 mod 7: {NumTheory.modpow(4,4,7)}, {4**4 % 7}' )

gcd(3,7): 1
gcd(21,7): 7
(1/3) mod 7: 5
3**3 mod 7: 6, 6
4**4 mod 7: 4, 4


### RSA algorithm

This calculates the decryption exponent from the encryption exponent, and does the 
powers that encrypt and decrypt.

The algorithm is powered by two distinct, odd primes $p$ and $q$ and an encryption exponent
that is relatively prime to $(p-1)(q-1)$.

The public information is $n=pq$ and the encryption exponent $e$. 
The private information is a decryption exponent $d$ that solves,
$$
d \,e = 1 \pmod{(p-1)(q-1)}
$$
Encryption of message $m$ is then,
$$
c = m^{e} \pmod{n}
$$
and decryption is
$$
m = c^{d} \pmod{n}
$$

In this form the algorithm has numerous problems as a practical encryption scheme, and needs to
be fixed up. However, we have here all the math you need to know.

In [11]:
   
class RSA_publickey:
    
    def __init__(self, p, q, encryption_exponent):
        
        assert(NumTheory.is_prime(p) and NumTheory.is_prime(q))
        assert(p!=q)
    
        self.n = p*q
        self.phi_n = (p-1)*(q-1)
        self.e_exp = encryption_exponent
        assert( math.gcd(self.e_exp,self.phi_n) )
        
        self.d_exp = NumTheory.modinv(self.e_exp,self.phi_n)
        
        assert ( (self.e_exp*self.d_exp) % self.phi_n == 1 )

    def __str__(self):
        return f'n: {self.n}, phi: {self.phi_n}, e: {self.e_exp}, d: {self.d_exp}'
        
    def encrypt(self,plaintext):
        assert NumTheory.egcd(plaintext,self.n)[0]==1, f'{plaintext} divides {self.n}'
        return NumTheory.modpow(plaintext,self.e_exp,self.n)
    
    def decrypt(self,ciphertext):
        return NumTheory.modpow(ciphertext,self.d_exp,self.n)

test = True
#test = False
if test:
    p,q =137,167
    e = 3
    rsa = RSA_publickey(p,q,e)
    print(rsa)
    assert (rsa.e_exp*rsa.d_exp % rsa.phi_n)==1
    
    tests = [1,7,100,1000,1234]
    for m in tests:
        c = rsa.encrypt(m)
        print(f'plaintext: {m}, ciphertext: {c}')
        assert rsa.decrypt(c)==m 

n: 22879, phi: 22576, e: 3, d: 15051
plaintext: 1, ciphertext: 1
plaintext: 7, ciphertext: 343
plaintext: 100, ciphertext: 16203
plaintext: 1000, ciphertext: 4668
plaintext: 1234, ciphertext: 5755
